## Задание 1

Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
ratings = pd.read_csv('ratings.dat', names = ['userId', 'movieId', 'rating', 'timestamp'], sep = '\s+', delimiter="::")
ratings.head()

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [113]:
time_live_df =  ratings.copy()
min_ratings = ratings.groupby('userId').min().reset_index()[['userId', 'timestamp']]
min_ratings.columns = ['userId', 'min']
max_ratings = ratings.groupby('userId').max().reset_index()[['userId', 'timestamp']]
max_ratings.columns = ['userId', 'max']
time_live_df = min_ratings.merge(max_ratings, how='inner', on='userId')
ratings_grouped = ratings.groupby('userId').count().reset_index()[['userId', 'rating']]
time_live_df = ratings_grouped[ ratings_grouped['rating'] >= 100].merge(time_live_df, how='left', on='userId')
time_live_df['time_live'] = time_live_df['max'] - time_live_df['min']
avg_live = datetime.fromtimestamp(time_live_df['time_live'].mean())
f"среднее время жизни пользователей, которые выставили более 100 оценок равно \
{avg_live.year-1970} лет \
{avg_live.month} месяцев \
{avg_live.day} дней \
{avg_live.minute} часов и \
{avg_live.second} минут \
"

'среднее время жизни пользователей, которые выставили более 100 оценок равно 0 лет 6 месяцев 12 дней 50 часов и 23 минут '

## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [88]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [89]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [90]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [91]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [95]:
all_transport = air.merge(rzd.merge(auto, how='outer', on='client_id'), how='outer', on='client_id')
all_transport

,client_id,air_revenue,rzd_revenue,auto_revenue
0,115,81.0,981.0,912.0
1,116,4.0,NaN,4834.0
2,117,13.0,NaN,98.0
3,118,173.0,NaN,NaN
4,111,NaN,1093.0,NaN
5,112,NaN,2810.0,NaN
6,113,NaN,10283.0,57483.0
7,114,NaN,5774.0,83.0


In [96]:
all_with_address = air.merge(rzd.merge(auto, how='outer', on='client_id'), how='outer', on='client_id').\
                        merge(client_base, how='left', on='client_id')
all_with_address

,client_id,air_revenue,rzd_revenue,auto_revenue,address
0,115,81.0,981.0,912.0,ЗЖБИиДК 1
1,116,4.0,NaN,4834.0,Строителей 18
2,117,13.0,NaN,98.0,Панфиловская 33
3,118,173.0,NaN,NaN,Мастеркова 4
4,111,NaN,1093.0,NaN,Комсомольская 4
5,112,NaN,2810.0,NaN,Энтузиастов 8а
6,113,NaN,10283.0,57483.0,Левобережная 1а
7,114,NaN,5774.0,83.0,Мира 14


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок?

Я бы завел еще одну таблицу, где хранились бы данные пользователей с широтой/долготой. Наверняка у всех трех таблиц есть поле время, которое надо учитывать при склеивании. Но т.к. время никогда не совпадет секунда-в-секунду, то надо хранить время "округленно", например вниз до ближайшей минуты. Тогда таблицы визиты-координаты и покупки-координта, надо склеивать по left и по паре полей id_клиента-время